In [67]:
import deepmatcher as dm
import pandas as pd
import numpy as np
from scipy.spatial import distance
import math

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
data_dir = 'sample_data/itunes-amazon'
datasets = dm.data.process(data_dir, train='train.csv', validation='validation.csv',
                           test='test.csv')
train = datasets[0]
validation = datasets[1]
test = datasets[2]

In [4]:
train_table = train.get_raw_table()
train_table.head()

,id,label,left_Song_Name,left_Artist_Name,left_Album_Name,left_Genre,left_Price,left_CopyRight,left_Time,left_Released,right_Song_Name,right_Artist_Name,right_Album_Name,right_Genre,right_Price,right_CopyRight,right_Time,right_Released
0,448,0,baby when the light ( david guetta & fred rist...,david guetta,pop life ( extended version ) [ bonus version ],"dance , music , rock , pop , house , electroni...",$ 1.29,‰ ãñ 2007 gum records,6:17,18-sep-07,revolver ( madonna vs. david guetta feat . lil...,david guetta,one love ( deluxe version ),dance & electronic,$ 1.29,( c ) 2014 swedish house mafia holdings ltd ( ...,3:18,"august 21 , 2009"
1,287,1,outversion,mark ronson,version,"pop , music , r & b / soul , soul , dance , ro...",$ 0.99,2007 mark ronson under exclusive license to so...,1:50,10-jul-07,outversion,mark ronson,version [ explicit ],pop,$ 0.99,( c ) 2011 j'adore records,1:50,"july 10 , 2007"
2,534,0,peer pressure ( feat . traci nelson ),snoop dogg,doggumentary,"hip-hop/rap , music , rock , gangsta rap , wes...",$ 1.29,"‰ ãñ 2011 capitol records , llc . all rights r...",4:07,29-mar-11,boom ( ( feat . t-pain ) [ edited ] ),snoop dogg,doggumentary [ edited ],"rap & hip-hop , west coast",$ 1.29,"( c ) 2011 capitol records , llc",3:50,"march 29 , 2011"
3,181,1,stars come out ( tim mason remix ),zedd,stars come out ( remixes ) - ep,"dance , music , electronic , house",$ 1.29,2012 dim mak inc .,5:49,20-may-14,stars come out ( dillon francis remix ),zedd,stars come out [ dillon francis remix ],dance & electronic,$ 1.29,2012 dim mak inc .,4:08,"may 20 , 2014"
4,485,0,jump ( feat . nelly furtado ),flo rida,r.o.o.t.s . ( deluxe version ),"hip-hop/rap , music",$ 1.29,‰ ãñ 2009 atlantic recording corporation for t...,3:28,30-mar-09,"yayo [ feat . brisco , billy blue , ball greez...",flo rida,r.o.o.t.s . ( route of overcoming the struggle...,rap & hip-hop,$ 1.29,"( c ) 2012 motown records , a division of umg ...",7:53,"march 30 , 2009"


## Define neural network models

In [5]:
hybrid_model = dm.MatchingModel(attr_summarizer='hybrid')

In [6]:
#run this cell if you want to load pre-trained model
hybrid_model.load_state('hybrid_model.pth')
hybrid_model.cuda()

MatchingModel(
  (attr_summarizers): ModuleMap(
    (Song_Name): Hybrid(
      (word_contextualizer): RNN(
        (rnn_groups): ModuleList(
          (0): GRU(300, 150, batch_first=True, bidirectional=True)
        )
        (dropouts): ModuleList(
          (0): Dropout(p=0)
        )
        (bypass_networks): ModuleList(
          (0): None
        )
        (input_dropout): NoMeta(
          (module): Dropout(p=0)
        )
      )
      (word_comparator): Attention(
        (alignment_networks): ModuleList(
          (0): AlignmentNetwork(
            (transform): Transform(
              (transforms): ModuleList(
                (0): Linear(in_features=300, out_features=300, bias=True)
                (1): Linear(in_features=300, out_features=300, bias=True)
              )
              (bypass_networks): ModuleList(
                (0): Bypass(
                  (highway_gate): Linear(in_features=300, out_features=300, bias=True)
                )
                (1): Bypass(


## Train models

In [ ]:
hybrid_model.run_train(
    train,
    validation,
    epochs=20,
    batch_size=16,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=5)

# Step 1 extract intermediate layers
In this step we want to evaluate the output of intermediate layers

## Classifier Evaluation
We want to evaluate the distance between positive and negative example respect to the classifier input

In [7]:
# class to save intermediate layer output/input
class Hook():
    def __init__(self, module, backward=False):
        if backward==False:
            self.hook = module.register_forward_hook(self.hook_fn)
        else:
            self.hook = module.register_backward_hook(self.hook_fn)
    def hook_fn(self, module, input, output):
        self.input = input
        self.output = output
    def close(self):
        self.hook.remove()

In [22]:
attributes = ['Song_Name','Artist_Name','Album_Name','Price','CopyRight','Released','Genre','Time']

In [ ]:
summarizers = []
comparators = []
for attr in attributes:
    summarizers.append(hybrid_model.attr_summarizers[attr])
    comparators.append(hybrid_model.attr_comparators[attr])

In [ ]:
hookF_summarizer = []
for summ in summarizers:
    hookF_summarizer.append(Hook(summ))

In [32]:
def return_layer_input_output(hook_functions,batch,model):
    out= model(batch)
    layer_inputs = []
    layer_outputs = []
    for hook in hook_functions:
        layer_inputs.append(hook.input)
        layer_outputs.append(hook.output)
    return layer_inputs,layer_outputs

In [ ]:
positives_batch_layer_inputs,positives_batch_layer_outputs = return_layer_input_output(hookF_summarizer,
                                                                                     positives_batches[0],hybrid_model)

In [ ]:
altered_batch_layer_inputs,altered_batch_layer_outputs = return_layer_input_output(hookF_summarizer,
                                                                                  altered_positives[0],hybrid_model)

In [ ]:
positives_summarizers_left_output = list(map(lambda x: x[0].data,positives_batch_layer_outputs))
positives_summarizers_right_output = list(map(lambda x: x[1].data,positives_batch_layer_outputs))

In [ ]:
altered_summarizers_left_output = list(map(lambda x: x[0].data,altered_batch_layer_outputs))
altered_summarizers_right_output = list(map(lambda x:x[1].data,altered_batch_layer_outputs))

In [ ]:
negative_classifier_inputs = list(map(lambda x: x[0][0].data,classifier_inputs))

In [ ]:
positives_classifier_input = positives_classifier_input[0][0].data

In [ ]:
distance.euclidean( positives_classifier_input[0],negative_classifier_inputs[1][27])

In [ ]:
altered_summarizers_right_output[0]

In [ ]:
def calculate_distance_matrix(summarizers_left_output,summarizers_right_output):
    distance_mat = []
    for i in range(len(summarizers_left_output)):
        distances = []
        for j in range(batch_size):
            l_out = summarizers_left_output[i][j].data
            r_out = summarizers_right_output[i][j].data
            dist = distance.euclidean(l_out,r_out)
            distances.append(dist)
        distance_mat.append(distances)
    distance_mat = np.matrix(distance_mat)
    return distance_mat

In [ ]:
distance_mat_positives = calculate_distance_matrix(positives_summarizers_left_output,positives_summarizers_right_output)
distance_mat_positives = distance_mat_positives.reshape((batch_size,8))

In [ ]:
distances_positives_df = pd.DataFrame(data = distance_mat_positives,columns=attributes)

In [ ]:
distances_positives_df.head(16)

In [ ]:
distances_positives_df.to_csv('distances_positives_batch1.csv',index=False)

In [ ]:
distance_mat_altered = calculate_distance_matrix(altered_summarizers_left_output,altered_summarizers_right_output)
distance_mat_altered = distance_mat_altered.reshape((batch_size,8))

In [ ]:
distances_altered_df = pd.DataFrame(data=distance_mat_altered,columns = attributes)

In [ ]:
distances_altered_df.head()

In [ ]:
distances_altered_df.to_csv('distances_altered_batch1.csv',index=False)

In [ ]:
differences_df = distances_positives_df.subtract(distances_altered_df)

In [ ]:
differences_df.head(16)

## Classifier Evaluation
We want to evaluate the distance between positive and negative example respect to the classifier input

In [25]:
classifier_datasets = dm.data.process(path='sample_data/itunes-amazon/',train='positive_samples.csv',
                            validation='negative_samples.csv',cache='pcache.pth')

In [26]:
from deepmatcher.data import MatchingIterator
batch_size = 32
splits = MatchingIterator.splits(classifier_datasets,batch_sizes=[batch_size,batch_size])

In [38]:
positive_batches = [] 
for batch in splits[0]:
    positive_batches.append(batch)

In [39]:
negative_batches = []
for batch in splits[1]:
    negative_batches.append(batch)

In [40]:
classifier = hybrid_model.classifier

In [41]:
hookF_classifier = []
hookF_classifier.append(Hook(classifier))

In [57]:
positive_classifier_inputs = []
positive_classifier_outputs = []
for batch in positive_batches:
    classifier_input,classifier_output = return_layer_input_output(hookF_classifier,batch,hybrid_model)
    positive_classifier_inputs.append(classifier_input)
    positive_classifier_outputs.append(classifier_output)

In [58]:
negative_classifier_inputs = []
negative_classifier_outputs = []
for batch in negative_batches:
    classifier_input,classifier_output = return_layer_input_output(hookF_classifier,batch,hybrid_model)
    negative_classifier_inputs.append(classifier_input)
    negative_classifier_outputs.append(classifier_output)

In [59]:
positive_classifier_inputs = list(map(lambda x: x[0][0].data,positive_classifier_inputs))
positive_classifier_outputs = list(map(lambda x: x[0][0].data,positive_classifier_outputs))

In [60]:
negative_classifier_inputs = list(map(lambda x: x[0][0].data,negative_classifier_inputs))
negative_classifier_outputs = list(map(lambda x: x[0][0].data,negative_classifier_outputs))

In [68]:
def euclidean_distance_with_max_difference_dimension(v,q):
    distance = 0
    max_difference = -1
    max_difference_dimension = 0
    for idx,vi in enumerate(v):
        qi = q[idx]
        diff = qi-vi
        if(diff>max_difference):
            max_difference = diff
            max_difference_dimension = idx
        distance += (diff)**2
    return math.sqrt(distance),max_difference_dimension

In [70]:
euclidean_distance_with_max_difference_dimension(positive_classifier_inputs[0][0],negative_classifier_inputs[0][20])

(18.899763763448036, 12)

In [ ]:
distance.euclidean(positive_classifier_inputs[0][0],)

In [ ]:
def calculate_closer_vector(pos_vectors,neg_vector_list):
    #mi salvo l'indice del vettore più vicino come chiave
    closer_vectors = []
    for curr_positive in pos_vectors:
        current_min = 100000000
        index = 1
        closer_index = -1
        max_dim = 0
        for batch in neg_vector_list:
            for curr_negative in batch:
                curr_distance,dim_max = euclidean_distance_with_max_difference_dimension(curr_positive,curr_negative)
                if(curr_distance<current_min):
                    current_min=curr_distance
                    closer_index = index
                    max_dim = dim_max
                index +=1
        closer_vectors.append((closer_index, current_min,dim_max))
    return closer_vectors

In [ ]:
closer_vectors = calculate_closer_vector(positives_classifier_input,negative_classifier_inputs)

In [ ]:
closer_vectors_df = pd.DataFrame(data=closer_vectors,columns=['Closer_vector','distance','max_difference_dimension'])

In [ ]:
closer_vectors_df.head(10)

In [ ]:
closer_vectors_df.to_csv('classifier_distances.csv',index=False)